# Installing Libraries & Dependencies

In [ ]:
#!pip3 install -r requirements.txt

In [ ]:
from datetime import datetime

# Input Arguments

Example of input arguments for the data validation component

In [ ]:
PROJECT = "irn-70656-dev-1307100302"
REGION = 'europe-west1'
BUCKET = "bike-sharing-pipeline-metadata"
BUCKET_STAGING = "bike-sharing-pipeline-staging"
PIPELINE_VERSION = "v0_1"
DATA_VERSION = "200909_154702"
MODEL_VERSION = datetime.now().strftime('%y%m%d_%H%M%S')
RUNNER = "AIplatformRunner" # DirectRunner or AIplatformRunner to run on AI platform

In [ ]:
TF_VERSION = '2.2'
PYTHON_VERSION = '3.7'

In [ ]:
# Set up some globals for gcs file
HANDLER = 'gs://' # ../ for local data, gs:// for cloud data

BASE_DIR = HANDLER + BUCKET+'/'+PIPELINE_VERSION
RUN_DIR = BASE_DIR+'/run/'+DATA_VERSION
DATA_DIR = RUN_DIR+'/data_transform'
OUTPUT_DIR = RUN_DIR+'/model_training/' + MODEL_VERSION

In [ ]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['BUCKET_STAGING'] = BUCKET_STAGING
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['PIPELINE_VERSION'] = PIPELINE_VERSION
os.environ['MODEL_VERSION'] = MODEL_VERSION

In [ ]:
os.environ['HANDLER'] = HANDLER
os.environ['RUNNER'] = RUNNER
os.environ['TF_VERSION'] = TF_VERSION
os.environ['PYTHON_VERSION'] = PYTHON_VERSION

In [ ]:
os.environ['DATA_DIR'] = DATA_DIR
os.environ['OUTPUT_DIR'] = OUTPUT_DIR

### If running on google cloud

Set up project and compute region.

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

# Move code into python package

Let's package our updated code with feature engineering so it's AI Platform compatible.

In [ ]:
%%bash
mkdir bikesharingmodel
mkdir bikesharingmodel/trainer
touch bikesharingmodel/trainer/__init__.py

In [ ]:
%%writefile bikesharingmodel/PKG-INFO
Metadata-Version: 1.0
Name: trainer
Version: 0.1
Summary: BikeSharingModel
Home-page: UNKNOWN
Author: MA MHANNA
Author-email: maggie.mhanna@renault.com
License: Apache
Description: UNKNOWN
Platform: UNKNOWN

In [ ]:
%%writefile bikesharingmodel/setup.cfg
[egg_info]
tag_build = 
tag_date = 0
tag_svn_revision = 0

In [ ]:
%%writefile bikesharingmodel/setup.py
from setuptools import find_packages, setup

NAME = 'trainer'
VERSION = '0.1'
AUTHOR = 'MA MHANNA'
EMAIL = 'maggie.mhanna@renault.com'

REQUIRED_PACKAGES = ['tensorflow==2.2.0', 
                     'tensorflow-cpu==2.2.0',
                     'tensorflow_transform==0.22.0',
                     'tensorflow_model_analysis==0.22.0', 
                     'apache_beam[gcp]==2.20.0', 
                     'pyarrow==0.16.0',
                     'tfx-bsl==0.22.0',
                     'absl-py==0.8.1']


setup(
    name=NAME,
    version=VERSION,
    author = AUTHOR,
    author_email = EMAIL,
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='Bike Sharing Demand Prediction.',
    requires=[]
)

# Create model.py

In [ ]:
%%writefile bikesharingmodel/model.py

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

# Features, labels, and key columns
NUMERIC_FEATURE_KEYS=["temp", "atemp", "humidity", "windspeed"] 
CATEGORICAL_FEATURE_KEYS=["season", "weather", "daytype"] 
KEY_COLUMN = "datetime"
LABEL_COLUMN = "count"

def transformed_name(key):
    return key 


def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(
      filenames,
      compression_type='GZIP')

def get_dataset_size(file_path):
    """Function that fetchs the size of the Tfrecords dataset."""
    size = 1
    file_list = tf.io.gfile.glob(file_path)
    for file in file_list:
        for record in tf.compat.v1.io.tf_record_iterator(file, options=tf.io.TFRecordOptions(
    compression_type='GZIP')):
            size += 1
    return size

def _get_serve_tf_examples_fn(model, label_column, tf_transform_output):
    """Returns a function that parses a serialized tf.Example and applies TFT."""

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(label_column)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        return model(transformed_features)

    return serve_tf_examples_fn

def _input_fn(file_pattern: List[Text],
              label_column,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 16) -> tf.data.Dataset:
    """Generates features and label for tuning/training.

    Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

    Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
    """
    transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())

    INPUT_KEYS = NUMERIC_FEATURE_KEYS + CATEGORICAL_FEATURE_KEYS + [LABEL_COLUMN]
    transformed_feature_spec = {key: transformed_feature_spec[key] for key in INPUT_KEYS}

    dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      label_key=transformed_name(label_column))
    
    return dataset

# Evaluation Metrics
def rmse(labels, predictions): # Root Mean Squared Error
    rmse = tf.sqrt(x = tf.reduce_mean(input_tensor = tf.square(x = labels - predictions)))
    return  rmse

def _build_keras_model(tf_transform_output: tft.TFTransformOutput,
                       hidden_units: List[int] = None,
                       learning_rate: float = 0.01) -> tf.keras.Model:
    """Creates a DNN Keras model for classifying taxi data.

    Args:
    hidden_units: [int], the layer sizes of the DNN (input layer first).

    Returns:
    A keras Model.
    """
    numeric_columns = [
      tf.feature_column.numeric_column(transformed_name(key), shape=())
      for key in NUMERIC_FEATURE_KEYS
    ]
    
    categorical_columns = [
      tf.feature_column.categorical_column_with_vocabulary_file(
        transformed_name(key), 
        vocabulary_file=tf_transform_output.vocabulary_file_by_name(
            vocab_filename=key), 
        dtype=tf.dtypes.string,
        default_value=None, 
        num_oov_buckets=0)
      for key in CATEGORICAL_FEATURE_KEYS
    ]
    
    indicator_columns = [
      tf.feature_column.indicator_column(categorical_column)
      for categorical_column in categorical_columns
    ]
        
    model = dnn_regressor(
      input_columns=numeric_columns + indicator_columns,
      dnn_hidden_units=hidden_units,
      learning_rate=learning_rate)
    return model


def dnn_regressor(input_columns, dnn_hidden_units, learning_rate):
    """Build a simple keras wide and deep model.

    Args:
    wide_columns: Feature columns wrapped in indicator_column for wide (linear)
      part of the model.
    deep_columns: Feature columns for deep part of the model.
    dnn_hidden_units: [int], the layer sizes of the hidden DNN.

    Returns:
    A Wide and Deep Keras model
    """
    # Following values are hard coded for simplicity in this example,
    # However prefarably they should be passsed in as hparams.


    input_layers = {
      colname: tf.keras.layers.Input(name=transformed_name(colname), shape=(), dtype=tf.float32)
      for colname in NUMERIC_FEATURE_KEYS
    }
    input_layers.update({
      colname: tf.keras.layers.Input(name=transformed_name(colname), shape=(), dtype='string')
      for colname in CATEGORICAL_FEATURE_KEYS
    })

    deep = tf.keras.layers.DenseFeatures(input_columns)(input_layers)
    for numnodes in dnn_hidden_units:
        deep = tf.keras.layers.Dense(numnodes, activation='relu')(deep)

    output = tf.keras.layers.Dense(
      1, activation=None)(deep)

    model = tf.keras.Model(input_layers, output)
    model.compile(
        optimizer = tf.keras.optimizers.Adam(lr=learning_rate),
        loss = "mean_squared_error",
        metrics=[rmse])
    model.summary(print_fn=absl.logging.info)
    return model


def train_and_evaluate(params):
    """Train the model based on given args.

    Args:
    params: Holds args used to train the model as name/value pairs.
    """
    
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO) # so loss is printed during training
    
    # Extract params from task.py
    DATA_DIR = params["data_dir"]
    OUTPUT_DIR = params["output_dir"]
    HIDDEN_UNITS_1 = params["hidden_units_1"]
    HIDDEN_UNITS_2 = params["hidden_units_2"]
    HIDDEN_UNITS_3 = params["hidden_units_3"]
    BATCH_SIZE = params["batch_size"]
    NUM_EPOCHS = params["num_epochs"]
    LEARNING_RATE = params["learning_rate"]
    
    # Setting up paths 
    TRAIN_PATH = DATA_DIR+'/train*'
    VAL_PATH = DATA_DIR+'/val*'
    TEST_PATH = DATA_DIR+'/test*'
    
    # Training set size
    TRAIN_SIZE = get_dataset_size(TRAIN_PATH)
    NUM_STEPS = TRAIN_SIZE / BATCH_SIZE # number of steps per epoch for which to train model
    
    tf_transform_output = tft.TFTransformOutput(os.path.join(DATA_DIR, 'tft_output'))
        
    train_dataset = _input_fn(TRAIN_PATH, LABEL_COLUMN, tf_transform_output, BATCH_SIZE)
    val_dataset = _input_fn(VAL_PATH, LABEL_COLUMN, tf_transform_output, BATCH_SIZE)

    model = _build_keras_model(
        tf_transform_output,
        hidden_units=[HIDDEN_UNITS_1, HIDDEN_UNITS_2, HIDDEN_UNITS_3],
        learning_rate=LEARNING_RATE)

    log_dir = os.path.join(OUTPUT_DIR, 'logs')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir, histogram_freq=1)
    
    # Setup Metric callback.
    class metric_cb(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            tf.summary.scalar('rmse', logs['rmse'], epoch) 
    
    model.fit(
      train_dataset,
      epochs=NUM_EPOCHS,        
      steps_per_epoch=NUM_STEPS,
      validation_data=val_dataset,
      validation_steps=10,
      callbacks=[tensorboard_callback, metric_cb()])

    signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    LABEL_COLUMN,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
    }
    model.save(OUTPUT_DIR, save_format='tf', signatures=signatures)

# Create task.py

In [ ]:
%%writefile bikesharingmodel/task.py
import argparse
import json
import os

from . import model

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--data_dir",
        help = "Location directory to read datasets",
        type = str,
        required = True
    )
    parser.add_argument(
        "--output_dir",
        help = "Location directory to write checkpoints and export models",
        type = str,
        required = True
    )
    parser.add_argument(
        "--hidden_units_1",
        help = "Hidden layer 1 sizes to use for DNN feature columns",
        type = int,
        default = 16
    )
    parser.add_argument(
        "--hidden_units_2",
        help = "Hidden layer 2 sizes to use for DNN feature columns",
        type = int,
        default = 16
    )
    parser.add_argument(
        "--hidden_units_3",
        help = "Hidden layer 3 sizes to use for DNN feature columns",
        type = int,
        default = 16
    )
    parser.add_argument(
        "--batch_size",
        help = "Mini-batch gradient descent batch size",
        type = int,
        default = 16
    )
    parser.add_argument(
        "--num_epochs",
        help = "Number of epochs",
        type = int,
        default = 10
    )
    parser.add_argument(
        "--learning_rate",
        help = "Hidden layer 1 sizes to use for DNN feature columns",
        type = int,
        default = 0.001
    )
    args = parser.parse_args().__dict__
    
    # Append trial_id to path so trials don"t overwrite each other
    args["output_dir"] = os.path.join(
        args["output_dir"],
        json.loads(
            os.environ.get("TF_CONFIG", "{}")
        ).get("task", {}).get("trial", "")
    ) 
        
    # Run the training job
    model.train_and_evaluate(args)

# Run locally

In [ ]:
%%bash
if [ "$RUNNER" = "DirectRunner" ]; then

    # Start fresh each time
    if ! [ "$HANDLER" = "gs://" ]; then
        rm -R ${OUTPUT_DIR}
    else
        gsutil -m rm -R ${OUTPUT_DIR}
    fi  

    python -m bikesharingmodel.task \
        --data_dir=${DATA_DIR} \
        --output_dir=${OUTPUT_DIR}
else 
echo "RUNNER = " $RUNNER
fi

# Running locally using gcloud

In [ ]:
%%bash
if [ "$RUNNER" = "DirectRunner" ]; then

    # Start fresh each time
    if ! [ "$HANDLER" = "gs://" ]; then
        rm -R ${OUTPUT_DIR}
    else
        gsutil -m rm -R ${OUTPUT_DIR}
    fi 

    # Use AI Platform to train the model in local file system
    gcloud ai-platform local train \
        --module-name=bikesharingmodel.task \
        --package-path=bikesharingmodel \
        -- \
        --data_dir=${DATA_DIR} \
        --output_dir=${OUTPUT_DIR}
else 
echo "RUNNER = " $RUNNER
fi

# TensorBoard 

In [ ]:
%load_ext tensorboard

In [ ]:
#%tensorboard --logdir $OUTPUT_DIR

# Run on AI Platform using gcloud

In [ ]:
%%writefile config.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: n1-standard-4
  evaluatorType: n1-standard-4
  evaluatorCount: 1

In [ ]:
%%bash
if [ "$RUNNER" = "AIplatformRunner" ]; then

    # Start fresh each time
    if ! [ "$HANDLER" = "gs://" ]; then
        rm -R ${OUTPUT_DIR}
    else
        gsutil -m rm -R ${OUTPUT_DIR}
    fi 

    JOB_NAME=${PIPELINE_VERSION}_${MODEL_VERSION}
    echo $OUTPUT_DIR $REGION $JOB_NAME

    gcloud ai-platform jobs submit training $JOB_NAME \
       --region=$REGION \
       --module-name=bikesharingmodel.task \
       --package-path=bikesharingmodel \
       --staging-bucket=gs://$BUCKET_STAGING \
       --config=config.yaml \
       --runtime-version=$TF_VERSION \
       --python-version=$PYTHON_VERSION \
       -- \
       --data_dir=$DATA_DIR \
       --output_dir=$OUTPUT_DIR 
else 
echo "RUNNER = " $RUNNER
fi

In [ ]:
%%bash
if [ "$RUNNER" = "AIplatformRunner" ]; then
    JOB_NAME=${PIPELINE_VERSION}_${MODEL_VERSION}
    gcloud ai-platform jobs describe $JOB_NAME
fi

# Run on AI Platform with Hyperparameter Tuning

AI Platform HyperTune, powered by Google Vizier, uses Bayesian Optimization by default, but also supports Grid Search and Random Search.

When tuning just a few hyperparameters (say less than 4), Grid Search and Random Search work well, but when tunining several hyperparameters and the search space is large Bayesian Optimization is best.

In [ ]:
%%writefile hyperparam.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: n1-standard-4
  evaluatorType: n1-standard-4
  evaluatorCount: 1
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 2
    maxParallelTrials: 2
    hyperparameterMetricTag: rmse
    params:
    - parameterName: hidden_units_1
      type: INTEGER
      minValue: 8
      maxValue: 64
      scaleType: UNIT_LOG_SCALE
    - parameterName: hidden_units_2
      type: INTEGER
      minValue: 8
      maxValue: 64
      scaleType: UNIT_LOG_SCALE
    - parameterName: hidden_units_3
      type: INTEGER
      minValue: 8
      maxValue: 64
      scaleType: UNIT_LOG_SCALE
    - parameterName: batch_size
      type: INTEGER
      minValue: 8
      maxValue: 64
      scaleType: UNIT_LOG_SCALE
    - parameterName: num_epochs
      type: INTEGER
      minValue: 1
      maxValue: 20
      scaleType: UNIT_LOG_SCALE

In [ ]:
%%bash
if [ "$RUNNER" = "AIplatformRunner" ]; then

    # Start fresh each time
    if ! [ "$HANDLER" = "gs://" ]; then
        rm -R ${OUTPUT_DIR}
    else
        gsutil -m rm -R ${OUTPUT_DIR}
    fi 

    JOB_NAME=${PIPELINE_VERSION}_${MODEL_VERSION}_hypertune
    echo $OUTPUT_DIR $REGION $JOBNAME

    gcloud ai-platform jobs submit training $JOB_NAME \
       --region=$REGION \
       --module-name=bikesharingmodel.task \
       --package-path=bikesharingmodel \
       --staging-bucket=gs://$BUCKET_STAGING \
       --runtime-version=$TF_VERSION \
       --python-version=$PYTHON_VERSION \
       --config=hyperparam.yaml \
       -- \
       --data_dir=$DATA_DIR \
       --output_dir=$OUTPUT_DIR  
else 
echo "RUNNER = " $RUNNER
fi

In [ ]:
%%bash
if [ "$RUNNER" = "AIplatformRunner" ]; then
    JOB_NAME=${PIPELINE_VERSION}_${MODEL_VERSION}_hypertune
    gcloud ai-platform jobs describe $JOB_NAME
fi